## 08) 엘모(Embeddings from Language Model, ELMo)

<img src = 'https://wikidocs.net/images/page/33930/elmo_DSHQjZD.png'>

논문 링크 : https://aclweb.org/anthology/N18-1202

ELMo(Embeddings from Language Model)는 2018년에 제안된 새로운 워드 임베딩 방법론이다. ELMo라는 이름은 세서미 스트리트라는 미국 인형극의 캐릭터 이름이기도 한데, 뒤에서 배우게 되는 BERT나 최근 마이크로소프트가 사용한 Big Bird라는 NLP 모델 또한 ELMo에 이어 세서미 스트리트의 캐릭터의 이름을 사용했다. ELMo는 Embeddings from Language Model의 약자이다. 해석하면 '언어 모델로 하는 임베딩'이다. ELMo의 가장 큰 특징은 **사전 훈련된 언어 모델(Pre-trained language model)**을 사용한다는 점이다. 이는 ELMo의 이름에 LM이 들어간 이유이다.

```
현재 텐서플로우 2.0에서는 TF-Hub의 ELMo를 사용할 수 없습니다. 사용하려면 텐서플로우 버전을 1버전으로 낮추어야 합니다. Colab에서 실습하시는 것을 권장드립니다. Colab에서는 손쉽게 텐서플로우 버전을 1버전으로 설정할 수 있습니다. 아래 실습 내용을 참고하세요.
```




### 1.ELMo(Embeddings from Language Model)

Bank라는 단어를 생각해보자. Bank Account(은행 계좌)와 River Bank(강둑)에서의 Bank는 전혀 다른 의미를 가지는데, Word2Vec이나 GloVe 등으로 표현된 임베딩 벡터들은 이를 제대로 반영하지 못한다는 단점이 있다. 예를 들어서 Word2Vec이나 GloVe 등의 임베딩 방법론으로 Bank란 단어를 [0.2 0.8 -1.2]라는 임베딩 벡터로 임베딩하였다고 하면, 이 단어는 Bank Account(은행 계좌)와 River Bank(강둑)에서의 Bank는 전혀 다른 의미임에도 불구하고 두 가지 상황 모두에서 [0.2 0.8 -1.2]의 벡터가 사용된다.

그렇다면 같은 표기의 단어라도 문맥에 따라서 다르게 워드 임베딩을 할 수 있으면 자연어 처리의 성능이 더 올라가지 않을까? 단어를 임베딩하기 전에 전체 문장을 고려해서 임베딩을 하겠다는 것이다. 그래서 탄생한 것이 **문맥을 반영한 워드 임베딩(Contextualized Word Embedding)**이다.


### 2.biLM(Bidirectional Language Model)의 사전 훈련

우선 다음 단어를 예측하는 작업인 언어 모델링을 상기해보자. 아래의 그림은 은닉층이 2개인 일반적인 단방향 RNN 언어 모델의 언어 모델링을 보여준다.

<img src = 'https://wikidocs.net/images/page/33930/deepbilm.PNG'>

RNN 언어 모델은 문장으로부터 단어 단위로 입력을 받는데, RNN 내부의 은닉 상태 $h_{t}$는 시점(time-step)이 지날수록 점점 업데이트 되어간다. 이는 결과적으로 $h_{t}$의 값이 문장의 문맥 정보를 점차적으로 반영한다고 말할 수 있다. 지금 설명하는 내용은 새로운 개념이 아니라 RNN의 기본 개념이다. 그런데 ELMo는 위의 그림의 순방향 RNN 뿐만 아니라, 위의 그림과는 반대 방향으로 문장을 스캔하는 역방향 RNN 또한 활용한다. ELMo는 양쪽 방향의 언어 모델을 둘 다 활용한다고하여 이 언어 모델을 **biLM(Bidirectional Language Model)**이라고 한다.

ELMo에서 말하는 biLM은 기본적으로 다층 구조(Multi-layer)를 전제로 한다. 은닉층이 최소 2개 이상이라는 의미이다. 아래의 그림은 은닉층이 2개인 순방향 언어 모델과 역방향 언어 모델의 모습을 보여준다.

<img src = 'https://wikidocs.net/images/page/33930/forwardbackwordlm2.PNG'>

이때 biLM의 입력이 되는 워드 임베딩 방법으로는 이 책에서는 다루지 않는 char CNN이라는 방법을 사용한다. 이 임베딩 방법은 글자(character) 단위로 계산되는데, 이렇게 하면 마치 서브단어(subword)의 정보를 참고하는 것처럼 문맥과 상관없이 dog란 단어와 doggy란 단어의 연관성을 찾아낼 수 있다. 또한 이 방법은 OOV에도 견고하다는 장점이 있다.

주의할 점은 앞서 RNN 챕터에서 설명한 **양방향 RNN**과 ELMo에서의 **biLM**은 다소 다르다. 양방향 RNN은 순방향 RNN의 은닉 상태와 역방향의 RNN의 은닉 상태를 다음 층의 입력으로 보내기 전에 연결(concatenate)시킨다. biLM의 순방향 언어모델과 역방향 언어모델이 각각의 은닉 상태만을 다음 은닉층으로 보내며 훈련시킨 후에 ELMo 표현으로 사용하기 위해서 은닉 상태를 연결(concatenate)시키는 것과는 다르다.



### 3.biLM의 활용

biLM이 훈련되었다면, 이제 ELMo가 사전 훈련된 biLM을 통해 입력 문장으로부터 단어를 임베딩하기 위한 과정을 보겠다.

<img src = 'https://wikidocs.net/images/page/33930/playwordvector.PNG'>

이 예제에서는 play란 단어가 임베딩이 되고 있다는 가정 하에 ELMo를 설명한다. play라는 단어를 임베딩하기 위해서 ELMo는 위의 점선의 사각형 내부의 각 층의 결과값을 재료로 사용한다. 다시 말해 해당 시점(time-step)의 biLM의 각 층의 출력값을 가져온다. 그리고 순방향 언어 모델과 역방향 언어 모델의 각 층의 출력값을 연결(concatenate)하고 추가 작업을 진행한다.

여기서 각 층의 출력값이란 첫번째는 임베딩 층을 말하며, 나머지 층은 각 층의 은닉 상태를 말한다. ELMo의 직관적인 아이디어는 각 층의 출력값이 가진 정보는 전부 서로 다른 종류의 정보를 갖고 있을 것이므로, 이들을 모두 활용한다는 점에 있다. 아래는 ELMo가 임베딩 벡터를 얻는 과정을 보여준다.



#### 1) 각 층의 출력값을 연결(concatenate)한다.

<img src = 'https://wikidocs.net/images/page/33930/concatenate.PNG'>



#### 2) 각 층의 출력값 별로 가중치를 준다.

<img src = 'https://wikidocs.net/images/page/33930/weight.PNG'>

이 가중치를 여기서는 $s_{1}, s_{2}, s_{3}$라고 하자.



#### 3) 각 층의 출력값을 모두 더한다.

<img src = 'https://wikidocs.net/images/page/33930/weightedsum.PNG'>

2)번과 3)번의 단계를 요약하여 가중합(Weighted Sum)을 한다고 할 수 있다.



#### 4) 벡터의 크기를 결정하는 스칼라 매개변수를 곱한다.

<img src = 'https://wikidocs.net/images/page/33930/scalarparameter.PNG'>

이 스칼라 매개변수를 여기서는 $γ$라고 하자.

이렇게 완성된 벡터를 ELMo 표현(representation)이라고 한다. 지금까지는 ELMo 표현을 얻기 위한 과정이고 이제 ELMo를 입력으로 사용하고 수행하고 싶은 텍스트 분류, 질의 응답 시스템 등의 자연어 처리 작업이 있을 것이다. 예를 들어 텍스트 분류 작업을 하고 싶다고 가정하자. 그렇다면 ELMo 표현을 어떻게 텍스트 분류 작업에 사용할 수 있을까?

ELMo 표현은 기존의 임베딩 벡터와 함께 사용할 수 있다. 우선 텍스트 분류 작업을 위해서 GloVe와 같은 기존의 방법론을 사용한 임베딩 벡터를 준비했다고 하자. 이 때, GloVe를 사용한 임베딩 벡터만 텍스트 분류 작업에 사용하는 것이 아니라 이렇게 준비된 ELMo 표현을 GloVe 임베딩 벡터와 연결(concatenate)해서 입력으로 사용할 수 있다. 그리고 이 때, ELMo 표현을 만드는데 사용되는 사전 훈련된 언어 모델의 가중치는 고정시킨다. 그리고 대신 위에서 사용한 $s_{1}, s_{2}, s_{3}$와 $γ$는 훈련 과정에서 학습된다.

<img src = 'https://wikidocs.net/images/page/33930/elmorepresentation.PNG'>

위의 그림은 ELMo 표현이 기존의 GloVe 등과 같은 임베딩 벡터와 함께 NLP 태스크의 입력이 되는 것을 보여준다.



### 4.ELMo 표현을 사용해서 스팸 메일 분류하기

이번 예제의 실습은 Colab에서 수행한다고 가정한다. 우선 시작 전에 텐서플로우 버전을 1버전으로 설정하겠다.


In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


텐서플로우 허브로부터 다양한 사전 훈련된 모델(Pre-trained Model)들을 사용할 수 있다. 여기서는 사전 훈련된 모델로부터 ELMo 표현을 사용해보는 정도로 예제를 진행해보겠다. 시작 전에 텐서플로우 허브를 인스톨해야 한다. 윈도우의 명령 프롬프트나 UNIX의 터미널에서 아래의 명령을 수행한다.


In [2]:
!pip install tensorflow-hub

설치가 끝났다면 이제 텐서플로우 허브를 임포트할 수 있다. 필요한 도구들을 임포트한다.


In [3]:
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K
import urllib.request
import pandas as pd
import numpy as np


텐서플로우 허브로부터 ELMo를 다운로드하겠다.


In [4]:
elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
# 텐서플로우 허브로부터 ELMo를 다운로드

sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

기본적으로 필요한 것들을 임포트하였다. 이제 데이터를 불러오고, 5개만 출력해보겠다.

파일 원본 출처 : https://www.kaggle.com/uciml/sms-spam-collection-dataset

스팸 메일 분류하기 데이터를 다운로드하겠다.



In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv",
                           filename="spam.csv")
data = pd.read_csv('spam.csv', encoding='latin-1')
data[:5]

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


여기서 필요한 건 v2열과 v1열이다. v1열은 숫자 레이블로 바꿔야 할 필요가 있다. 이를 각각 X_data와 y_data로 저장하겠다.


In [6]:
data['v1'] = data['v1'].replace(['ham', 'spam'], [0, 1])
y_data = list(data['v1'])
X_data = list(data['v2'])

v2열을 X_data에 저장한다. v1열에 있는 ham과 spam 레이블을 각각 숫자 0과 1로 바꾸고 y_data에 저장한다. 정상적으로 저장되었는지 이를 각각 5개만 출력해보겠다.


In [7]:
X_data[:5]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though"]

In [8]:
print(y_data[:5])

[0, 0, 1, 0, 0]


훈련 데이터와 테스트 데이터를 8:2 비율로 분할해보겠다. 그런데 그 전에 이를 위해 전체 데이터 개수의 80%와 20%는 각각 몇 개인지 확인한다.


In [9]:
print(len(X_data))
n_of_train = int(len(X_data) * 0.8)
n_of_test = int(len(X_data) - n_of_train)
print(n_of_train)
print(n_of_test)

5572
4457
1115


전체 데이터는 5,572개이며 8:2로 비율하면 4,457과 1,115가 된다. 이를 각각 훈련 데이터와 테스트 데이터의 양으로 하여 데이터를 분할하겠다. 



In [10]:
X_train = np.asarray(X_data[:n_of_train]) #X_data 데이터 중에서 앞의 4457개의 데이터만 저장
y_train = np.asarray(y_data[:n_of_train]) #y_data 데이터 중에서 앞의 4457개의 데이터만 저장

X_test = np.asarray(X_data[n_of_train:]) #X_data 데이터 중에서 뒤의 1115개의 데이터만 저장
y_test = np.asarray(y_data[n_of_train:]) #y_data 데이터 중에서 뒤의 1115개의 데이터만 저장


이제 훈련을 위한 데이터 준비는 끝났다. 이제 ELMo와 설계한 모델을 연결하는 작업들을 진행해보겠다. ELMo는 텐서플로우 허브로부터 가져온 것이기 때문에 케라스에서 사용하기 위해서는 케라스에서 사용할 수 있도록 변환해주는 작업들이 필요하다.


In [11]:
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict = True,
                           signature = 'default')['default']
# 데이터의 이동이 케라스 → 텐서플로우 → 케라스가 되도록 하는 함수

이제 모델을 설계한다.


In [12]:
from keras.models import Model
from keras.layers import Dense, Lambda, Input

input_text = Input(shape = (1, ), dtype = tf.string)
embedding_layer = Lambda(ELMoEmbedding, output_shape = (1024, ))(input_text)
hidden_layer = Dense(256, activation = 'relu')(embedding_layer)
output_layer = Dense(1, activation = 'sigmoid')(hidden_layer)

model = Model(inputs = [input_text], outputs = output_layer)
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 262,657
Trainable params: 262,657
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


모델은 ELMo를 이용한 임베딩 층을 거쳐서 256개의 뉴런이 있는 은닉층을 거친 후 마지막 1개의 뉴런을 통해 이진 분류를 수행한다. 이진 분류를 위한 마지막 뉴런의 활성화 함수는 시그모이드 함수이며, 모델의 손실 함수는 binary_crossentropy이다.



In [14]:
history = model.fit(X_train, y_train,
                    epochs = 1,
                    batch_size = 60)

Epoch 1/1
4457/4457 [==============================] - 1239s 278ms/step - loss: 0.0965 - accuracy: 0.9666


훈련 데이터에서는 정확도 96%를 얻었다. 이제 테스트 데이터에 대해서 평가해보겠다.


In [15]:
print('\n 테스트 정확도: %.4f' % (model.evaluate(X_test, y_test)[1]))

1115/1115 [==============================] - 295s 265ms/step

 테스트 정확도: 0.9812


1번의 에포크에서 테스트 데이터 정확도 98%를 얻어낸다.



참고 자료 :

http://www.realworldnlpbook.com/blog/improving-sentiment-analyzer-using-elmo.html

https://createmomo.github.io/2018/01/23/Super-Machine-Learning-Revision-Notes/

https://www.analyticsvidhya.com/blog/2019/03/learn-to-use-elmo-to-extract-features-from-text/

http://www.davidsbatista.net/blog/2018/12/06/Word_Embeddings/

https://lilianweng.github.io/lil-log/2019/01/31/generalized-language-models.html#bidirectional-language-model

https://medium.com/saarthi-ai/elmo-for-contextual-word-embedding-for-text-classification-24c9693b0045

char CNN 참고 자료 :

https://www.slideshare.net/JaeminCho6/dl-chatbot-seminar-day-02

한국어에 대한 ELMo :

https://github.com/HIT-SCIR/ELMoForManyLangs

https://ratsgo.github.io/embedding/

ELMo로 단어 몇 개만 임베딩 벡터 얻어보기 :

https://medium.com/@joeyism/embedding-with-tensorflow-hub-in-a-simple-way-using-elmo-d1bfe0ada45c

https://github.com/strongio/keras-elmo/blob/master/Elmo%20Keras.ipynb